## 2. ChatGPT API (1990~2019 Sampling 후, Zero-Shot)

In [58]:
%%writefile /content/news-classifier/notebooks/03_eval_on_gpt_labels.ipynb
# ============================================================================
# 1. 필수 패키지 설치 및 라이브러리 임포트
# ============================================================================
print("="*80)
print("🔧 1. 필수 패키지 설치 및 라이브러리 임포트")
print("="*80)
!pip install -q --upgrade transformers datasets accelerate scikit-learn polars pandas
print("✅ 패키지 설치 완료\n")

import os
import sys
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import gc
from google.colab import drive
from tqdm.auto import tqdm

# Scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Hugging Face
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# TQDM이 Pandas와 잘 작동하도록 설정
tqdm.pandas()

# ============================================================================
# 2. Google Drive 마운트 및 훈련된 모델/3요소 로드
# ============================================================================
print("\n" + "="*80)
print("📁 2. Google Drive 마운트 및 훈련된 모델/3요소 로드")
print("="*80)

drive.mount('/content/drive', force_remount=False)

# 훈련 시 11번 섹션에서 저장했던 경로
SAVE_BASE_PATH = Path("/content/drive/MyDrive/best_news_classifier")
if not SAVE_BASE_PATH.exists():
    raise FileNotFoundError(f"❌ 저장된 모델 경로를 찾을 수 없습니다: {SAVE_BASE_PATH}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 사용 디바이스: {device}")

try:
    # 1) 딥러닝(DL) 모델 로드
    print("... 1/5) 딥러닝(RoBERTa) 모델 로드 중 ...")
    dl_model = AutoModelForSequenceClassification.from_pretrained(SAVE_BASE_PATH).to(device)
    print("✅ DL 모델 로드 완료")

    # 2) 토크나이저 로드
    print("... 2/5) 토크나이저 로드 중 ...")
    tokenizer = AutoTokenizer.from_pretrained(SAVE_BASE_PATH)
    print("✅ 토크나이저 로드 완료")

    # 3) 라벨 인코더 로드 (DL, LR 공통 사용)
    print("... 3/5) 라벨 인코더 로드 중 ...")
    with open(SAVE_BASE_PATH / "label_encoder.pkl", 'rb') as f:
        label_encoder = pickle.load(f)
    print(f"✅ 라벨 인코더 로드 완료 (클래스: {label_encoder.classes_})")

    # 4) 로지스틱 회귀(LR) 모델 로드
    print("... 4/5) 로지스틱 회귀(LR) 모델 로드 중 ...")
    with open(SAVE_BASE_PATH / "baseline_lr.pkl", 'rb') as f:
        lr_clf = pickle.load(f)
    print("✅ LR 모델 로드 완료")

    # 5) TF-IDF 벡터라이저 로드
    print("... 5/5) TF-IDF 벡터라이저 로드 중 ...")
    with open(SAVE_BASE_PATH / "baseline_tfidf_vec.pkl", 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    print("✅ TF-IDF 벡터라이저 로드 완료")

except FileNotFoundError as e:
    print(f"\n❌ [오류] 모델 파일 로드 실패: {e}")
    print("이전 학습(11번 섹션)이 완료되었는지, 경로가 올바른지 확인하세요.")
    sys.exit(1)


# ============================================================================
# 3. [★평가 전용★] ChatGPT로 라벨링된 샘플 파일 로드
# ============================================================================
print("\n" + "="*80)
print("🚀 3. ChatGPT 라벨링 샘플 파일 로드")
print("="*80)

# ⚠️⚠️⚠️ [사용자 수정 필요] ⚠️⚠️⚠️
# 라벨링 작업(500, 1000, 3000개...)을 통해 최종 저장된 parquet 파일 경로
LABELED_DATA_PATH = SAVE_BASE_PATH / "chatgpt_1000_labels_1990_2019.parquet"
# (만약 500개 버전을 쓰려면: "chatgpt_500_labels_1990_2019.parquet")
# (만약 3000개 버전을 쓰려면: "chatgpt_3000_labels_1990_2019.parquet")

if not LABELED_DATA_PATH.exists():
     raise FileNotFoundError(f"❌ 라벨링된 샘플 파일({LABELED_DATA_PATH.name})을 찾을 수 없습니다.")

print(f"✅ {LABELED_DATA_PATH.name} 파일을 로드합니다.")
sample_df = pd.read_parquet(LABELED_DATA_PATH)
print(f"✅ 총 {len(sample_df)}개 샘플 로드 완료.")

# (API 호출, 필터링, 샘플 추출, 저장 로직 모두 제거됨)


# ============================================================================
# 4. [★평가 전용★] LR / DL 모델로 샘플 추론 실행
# ============================================================================
print("\n" + "="*80)
print(f"🔥 4. LR / DL 모델로 {len(sample_df)}개 샘플 추론 실행")
print("="*80)

# --- 4-1. 로지스틱 회귀(LR) 추론 ---
print("... 1/2) 로지스틱 회귀(LR) 추론 중 ...")
# 훈련(5-4)과 100% 동일한 전처리: [헤드라인 + " " + 본문][:1000]
X_text_lr = sample_df.apply(
    lambda row: f"{row['headline'] if row['headline'] else ''} {row['content'] if row['content'] else ''}"[:1000],
    axis=1
)
X_tfidf_lr = tfidf_vectorizer.transform(X_text_lr)
lr_pred_indices = lr_clf.predict(X_tfidf_lr)
lr_pred_labels = label_encoder.inverse_transform(lr_pred_indices)
sample_df['lr_label'] = lr_pred_labels
print("✅ LR 추론 완료")

# --- 4-2. 딥러닝(DL) 추론 ---
print("... 2/2) 딥러닝(RoBERTa) 추론 중 ...")
sample_hf_dataset = Dataset.from_pandas(sample_df)

def preprocess_and_tokenize_inference(examples):
    h = [t if t else "" for t in examples['headline']]
    c = [t if t else "" for t in examples['content']]
    texts = [f"{hh} {cc}"[:1000] for hh, cc in zip(h, c)]
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

N_PROC = 4 # 샘플 수가 적으므로 4로도 충분
sample_ds_tokenized = sample_hf_dataset.map(preprocess_and_tokenize_inference, batched=True, num_proc=N_PROC, remove_columns=sample_hf_dataset.column_names)
sample_ds_tokenized.set_format("torch")

# L4 또는 A100에 맞는 추론 설정
training_args = TrainingArguments(
    output_dir="/content/temp_inference_output",
    per_device_eval_batch_size=96,  # L4(24GB) 또는 A100(80GB)에서 넉넉한 배치
    fp16=True,
    dataloader_num_workers=4,
    report_to="none"
)
trainer = Trainer(model=dl_model, args=training_args)

dl_predictions_output = trainer.predict(sample_ds_tokenized)
dl_pred_indices = np.argmax(dl_predictions_output.predictions, axis=1)
dl_pred_labels = label_encoder.inverse_transform(dl_pred_indices)
sample_df['dl_label'] = dl_pred_labels
print("✅ DL 추론 완료")


# ============================================================================
# 5. [★평가 전용★] 최종 성능 비교
# ============================================================================
print("\n" + "="*80)
print(f"📊 5. 최종 성능 비교 ({len(sample_df)}개 샘플 기준)")
print("="*80)

valid_samples = sample_df[
    (sample_df['chatgpt_label'] != "분류실패") &
    (sample_df['chatgpt_label'] != "분류오류")
].copy()

print(f"총 {len(sample_df)}개 샘플 중 {len(valid_samples)}개의 유효한 샘플로 비교를 진행합니다.\n")

if len(valid_samples) > 0:
    lr_accuracy = accuracy_score(
        valid_samples['chatgpt_label'],
        valid_samples['lr_label']
    )
    dl_accuracy = accuracy_score(
        valid_samples['chatgpt_label'],
        valid_samples['dl_label']
    )

    print(f"--- 1990-2019 데이터 (GPT-4o-mini 기준) ---")
    print(f"🤖 [로지스틱 회귀] 정확도: {lr_accuracy:.2%}")
    print(f"🔥 [딥러닝 RoBERTa] 정확도: {dl_accuracy:.2%}")
else:
    print("❌ 비교할 유효한 ChatGPT 분류 샘플이 없습니다.")

print("\n🎉 모든 평가 작업 완료! 🎉")
print("--- [최종 결과 샘플] ---")
print(valid_samples[['headline', 'chatgpt_label', 'lr_label', 'dl_label']].head())

# (최종 결과 저장 로직은 이미 라벨링 스크립트에 있으므로, 여기서는 생략하거나 덮어쓰기)
FINAL_COMPARISON_PATH = SAVE_BASE_PATH / f"final_{len(sample_df)}_comparison_result.parquet"
valid_samples.to_parquet(FINAL_COMPARISON_PATH, index=False)
print(f"\n💾 최종 비교 결과가 {FINAL_COMPARISON_PATH.name} 에 저장되었습니다.")

Writing /content/news-classifier/notebooks/03_eval_on_gpt_labels.ipynb
